In [2]:
import requests as rq
import pandas as pd
import re

In [15]:
urlsessions=r'https://f1.tfeed.net/sessions/eng_sessions.js'

seasons=rq.get(urlsessions)
teams=pd.Series(re.findall("'\d\d','\D+','\d{4}_\D+','.+','(.+)'",seasons.text.replace('\t',''))).unique()
teams_dict={}
for team in teams:
    teams_dict[team]=rq.get('https://f1.tfeed.net/teams/eng_{}.js'.format(team)).text

drivers=pd.Series(re.findall("'\d\d','\D+','\d{4}_\D+','(.+)',.+",seasons.text.replace('\t',''))).unique()
drivers_dict={}
for driver in drivers:
    drivers_dict[driver]=rq.get('https://f1.tfeed.net/drivers/eng_{}.js'.format(driver)).text


races=re.findall('(\d{4}_\D+)\',',seasons.text)
#season data to get combinations of drivers, teams, etc
season_data=re.findall('new Array\((.+)\);',seasons.text.replace('\n',''))
season_data=pd.DataFrame(season_data[0].split('\t'))
season_data[0]=season_data[0].apply(lambda x: re.sub('\)?;.+Array\(','',x))
season_data[0]=season_data[0].apply(lambda x: re.sub('//','',x))

to_drop=season_data[season_data[0]=='']
season_data[0]=season_data[0].drop(to_drop.index)
season_data.dropna(inplace=True)
season_data.reset_index(inplace=True,drop=True)
season_data[0]=season_data[0].str.replace("'","")
season_data=season_data[0].str.split(',',expand=True).drop(columns=[5])
season_data['season']=season_data[3].apply(lambda x: re.findall('(\d{4})',x)[0])
season_data.rename(columns={0:'weekend_num',1:'race_name',2:'session_name',3:'driver_set',4:'team_set'},inplace=True)
season_data['year']=season_data['session_name'].apply(lambda x: re.findall('\d+',x)[0])

In [48]:
season_data['notes']=None
notes_raw=re.findall('notes\[.+\]=.+;',seasons.text.replace('\n',''))[0].split(';')
notes=[]
[notes.append(x) for x in notes_raw if 'notes' in x]

for x in notes:
    test=re.findall("'(\d+)'",x)
    id=season_data.index[(season_data['year']==test[0])&(season_data['weekend_num']==test[1])][0]
    season_data.loc[id,'notes']=re.findall('"(.+)"',x)[0]


In [44]:
import pickle
dir=r'../data/processed/f1-tfeed'
# save dictionary to session_data.pkl file
with open(dir+r'\team_data.pkl', 'wb') as fp:
    pickle.dump(teams_dict, fp)
    print('dictionary saved successfully to file')
with open(dir+r'\drivers_data.pkl', 'wb') as fp:
    pickle.dump(drivers_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file


In [30]:
import re
#racesx=re.findall('(\d{4}_\D+)\',',x.text)
races=pd.read_csv(r'..\Data\f1-tfeed\season_data.csv')['session_name']
sessions={}

In [88]:
races=season_data['session_name']
sessions={}
for num,formats in enumerate(races):

    url=r'https://f1.tfeed.net/sessions/{}/session.js?'.format(formats)
    if (formats not in list(sessions.keys())) and (formats not in ['2021_spa','2023_imola']):
        resp=rq.get(url)
        if resp.status_code!=200:
            i=0 
            while resp.status_code!=200:
                print(f'Retrying {formats}, trial number {i+1}...')
                resp=rq.get(url)
                i+=1
            continue
        sessions[formats]=resp.text
    else: continue
        

Retrying 2024_suzuka, trial number 1...
Retrying 2024_suzuka, trial number 2...
Retrying 2024_suzuka, trial number 3...
Retrying 2024_suzuka, trial number 4...
Retrying 2024_suzuka, trial number 5...
Retrying 2024_suzuka, trial number 6...
Retrying 2024_suzuka, trial number 7...
Retrying 2024_suzuka, trial number 8...
Retrying 2024_suzuka, trial number 9...
Retrying 2024_suzuka, trial number 10...
Retrying 2024_suzuka, trial number 11...
Retrying 2024_suzuka, trial number 12...
Retrying 2024_suzuka, trial number 13...
Retrying 2024_suzuka, trial number 14...
Retrying 2024_suzuka, trial number 15...
Retrying 2024_suzuka, trial number 16...
Retrying 2024_suzuka, trial number 17...
Retrying 2024_suzuka, trial number 18...
Retrying 2024_suzuka, trial number 19...
Retrying 2024_suzuka, trial number 20...
Retrying 2024_suzuka, trial number 21...
Retrying 2024_suzuka, trial number 22...
Retrying 2024_suzuka, trial number 23...
Retrying 2024_suzuka, trial number 24...
Retrying 2024_suzuka, tri

KeyboardInterrupt: 

['2009_singapore',
 '2009_suzuka',
 '2009_brazil',
 '2009_abu_dhabi',
 '2010_bahrain',
 '2010_australia',
 '2010_malaysia',
 '2010_china',
 '2010_spain',
 '2010_monaco',
 '2010_turkey',
 '2010_canada',
 '2010_valencia',
 '2010_silverstone',
 '2010_hockenheim',
 '2010_hungary',
 '2010_spa',
 '2010_monza',
 '2010_singapore',
 '2010_suzuka',
 '2010_korea',
 '2010_brazil',
 '2010_abu_dhabi',
 '2011_australia',
 '2011_malaysia',
 '2011_china',
 '2011_turkey',
 '2011_spain',
 '2011_monaco',
 '2011_canada',
 '2011_valencia',
 '2011_silverstone',
 '2011_nurburgring',
 '2011_hungary',
 '2011_spa',
 '2011_monza',
 '2011_singapore',
 '2011_suzuka',
 '2011_korea',
 '2011_india',
 '2011_abu_dhabi',
 '2011_brazil',
 '2012_australia',
 '2012_malaysia',
 '2012_china',
 '2012_bahrain',
 '2012_spain',
 '2012_monaco',
 '2012_canada',
 '2012_valencia',
 '2012_silverstone',
 '2012_hockenheim',
 '2012_hungary',
 '2012_spa',
 '2012_monza',
 '2012_singapore',
 '2012_suzuka',
 '2012_korea',
 '2012_india',
 '20

In [34]:
import pickle
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'
# save dictionary to session_data.pkl file
with open(dir+'\session_data.pkl', 'wb') as fp:
    pickle.dump(sessions, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [35]:
import pickle
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'
# Read dictionary pkl file
with open(dir+'\session_data.pkl', 'rb') as fp:
    sessions = pickle.load(fp)
    

In [ ]:
import os
import zipfile
from tqdm import tqdm
dir=r'../data/interim/f1-tfeed'

errors=[]
for formats in races:
    retries=0
    
    url=r'https://f1.tfeed.net/sessions/{}/session.zip'.format(formats)
    if os.path.exists(dir+'\{}'.format(formats)):
        continue
    print('\nDownloading '+formats)
    os.mkdir(dir+'\{}'.format(formats))
    while retries<3:
        try:
            print('Download attempt number '+str(retries+1))
            chunkSize = 1024
            r = rq.get(url, stream=True)
            with open(dir+'\{}\session.zip'.format(formats), 'wb') as f:
                pbar = tqdm( unit="B", total=int( r.headers['Content-Length'] ) )
                for chunk in r.iter_content(chunk_size=chunkSize): 
                    if chunk: # filter out keep-alive new chunks
                        pbar.update (len(chunk))
                        f.write(chunk)
            with zipfile.ZipFile(dir+'\{}\session.zip'.format(formats), 'r') as zip_ref:
                print('Extracting zip...')
                for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
                    zip_ref.extract(member=file, path=dir+'\{}'.format(formats))
            break
        except:
            retries+=1
            raise e
    if retries==3:
        print('Download {} failed'.format(formats))
        errors.append(formats)
    else:
        print('Download {} success'.format(formats))

In [ ]:
#todo: process semua link
#https://f1.tfeed.net/sessions/2024_calendar.js? - kalender
#https://f1.tfeed.net/sessions/config.js?v=10.7 - konfigurasi skor, dan rulebook

In [142]:
import json
urlcalendar=r'https://f1.tfeed.net/sessions/2024_calendar.js?'
texts=re.findall('calendar=new Array\((.+);',rq.get(urlcalendar).text.replace('\n','').replace('\t',''))[0]
dfSchedule=pd.DataFrame(re.findall("'(\d+)','(\d+/\d+/\d+)','(\d+:\d+)',cst(Practice|Qualification|Sprint|Race),",texts))#.drop(0,axis=1)
dfSchedule[1]=dfSchedule[1].str.replace('31/09','30/09')
dfSchedule['TS']=pd.to_datetime(dfSchedule[1]+' '+dfSchedule[2],format='%d/%m/%Y %H:%M')



Timestamp('2024-02-29 14:30:00')

In [144]:
import pytz
dfSchedule['TS'][0].replace(tzinfo=pytz.timezone('Africa/Abidjan'))

Timestamp('2024-02-29 14:30:00+0000', tz='Africa/Abidjan')